In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [2]:
df = pd.read_csv('anki_revlog.csv')
df.head()

,id,cid,ease,lastIvl,type
0,1381391111515,1381391091222,1,-60,0
1,1381391118078,1381391091222,1,-60,0
2,1381392124223,1381391091222,3,-60,0
3,1381392128210,1381391091222,3,-600,0
4,1381444213315,1381391091223,1,-60,0


In [3]:
df = df.drop(df[df['ease']==0].index)

In [4]:
df['raw_date'] = pd.to_datetime(df['id'],unit='ms')
df['raw_date'] = df['raw_date'].dt.tz_localize('UTC').dt.tz_convert('Asia/Shanghai')
df.head()

,id,cid,ease,lastIvl,type,raw_date
0,1381391111515,1381391091222,1,-60,0,2013-10-10 15:45:11.515000+08:00
1,1381391118078,1381391091222,1,-60,0,2013-10-10 15:45:18.078000+08:00
2,1381392124223,1381391091222,3,-60,0,2013-10-10 16:02:04.223000+08:00
3,1381392128210,1381391091222,3,-600,0,2013-10-10 16:02:08.210000+08:00
4,1381444213315,1381391091223,1,-60,0,2013-10-11 06:30:13.315000+08:00


In [5]:
df['date'] = df['raw_date'].map(lambda x: x-timedelta(days=1) if x.hour < 5 else x)
df['date'] = df['date'].dt.floor('D')
df.drop(df[df['date'].dt.year < 2017].index, inplace=True)
df.drop_duplicates(['cid', 'date'], keep='first', inplace=True)
df.head()

,id,cid,ease,lastIvl,type,raw_date,date
1727,1492836849841,1482314752476,4,-60,0,2017-04-22 12:54:09.841000+08:00,2017-04-22 00:00:00+08:00
1728,1492836858505,1482399896125,4,-60,0,2017-04-22 12:54:18.505000+08:00,2017-04-22 00:00:00+08:00
1729,1492836863457,1482394049701,4,-60,0,2017-04-22 12:54:23.457000+08:00,2017-04-22 00:00:00+08:00
1730,1492836871729,1482316716328,1,-60,0,2017-04-22 12:54:31.729000+08:00,2017-04-22 00:00:00+08:00
1731,1492836876449,1482392528727,4,-60,0,2017-04-22 12:54:36.449000+08:00,2017-04-22 00:00:00+08:00


In [6]:
df.sort_values(by=['cid', 'id'], inplace=True)
df['used_interval'] = df.date.diff().dt.days
df.loc[df['type']==0, 'used_interval'] = 0

In [7]:
for cid in df.loc[df['used_interval']<0, 'cid']:
    df.drop(df[df.cid == cid].index, inplace=True)

In [8]:
df['p'] = df['ease'].map({1:0, 2:1, 3:1, 4:1})

In [9]:
del df['id']
del df['raw_date']
del df['date']

In [10]:
df['acc_rep'] = 0
df['acc_lapses'] = 0
df['con_rep'] = 0
df['con_lapses'] = 0
df['repeat_long'] = 1
df.head()

,cid,ease,lastIvl,type,used_interval,p,acc_rep,acc_lapses,con_rep,con_lapses,repeat_long
3390,1321762260687,4,-60,0,0.0,1,0,0,0,0,1
3391,1321762260688,4,-60,0,0.0,1,0,0,0,0,1
3606,1321762260705,1,-60,0,0.0,0,0,0,0,0,1
3431,1321762260709,3,-60,0,0.0,1,0,0,0,0,1
3680,1321762260729,4,-60,0,0.0,1,0,0,0,0,1


In [11]:
cols = ['cid', 'type', 'repeat_long', 'acc_lapses', 'con_lapses', 'acc_rep', 'con_rep', 'used_interval', 'lastIvl', 'ease', 'p']
df = df[cols]
df.head()

,cid,type,repeat_long,acc_lapses,con_lapses,acc_rep,con_rep,used_interval,lastIvl,ease,p
3390,1321762260687,0,1,0,0,0,0,0.0,-60,4,1
3391,1321762260688,0,1,0,0,0,0,0.0,-60,4,1
3606,1321762260705,0,1,0,0,0,0,0.0,-60,1,0
3431,1321762260709,0,1,0,0,0,0,0.0,-60,3,1
3680,1321762260729,0,1,0,0,0,0,0.0,-60,4,1


In [12]:
def get_feature(x):
    for idx, log in enumerate(x.itertuples()):
        if idx == x.shape[0] - 1:
            break
        x.iloc[idx + 1, 2] = x.iloc[idx, 2] + 1
        if log.p == 1:
            x.iloc[idx + 1, 3] = x.iloc[idx, 3]
            x.iloc[idx + 1, 4] = 0
            x.iloc[idx + 1, 5] = x.iloc[idx, 5] + 1
            x.iloc[idx + 1, 6] = x.iloc[idx, 6] + 1
        else:
            x.iloc[idx + 1, 3] = x.iloc[idx, 3] + 1
            x.iloc[idx + 1, 4] = x.iloc[idx, 4] + 1
            x.iloc[idx + 1, 5] = x.iloc[idx, 5]
            x.iloc[idx + 1, 6] = 0
    return x
df = df.groupby('cid').apply(get_feature)

In [13]:
df.to_csv('revlog_clear.csv', index=False)